<a href="https://colab.research.google.com/github/sasansharifipour/Spark_Class/blob/main/SPARK_CC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

import findspark
findspark.init()

In [120]:
file_download_link = "https://drive.google.com/u/0/uc?id=1BgY3iHSZ0ImXc6atr61YQRcloxDwNrvi&export=download" 
!wget -O paper_graph_adj_list.txt --no-check-certificate "$file_download_link"

file_download_link = "https://drive.google.com/u/0/uc?id=1qmwKYipNwJZyheNfVxW8GwzhbY6J2nPU&export=download" 
!wget -O paper_graph_node_list.txt --no-check-certificate "$file_download_link"

--2021-04-25 11:07:51--  https://drive.google.com/u/0/uc?id=1BgY3iHSZ0ImXc6atr61YQRcloxDwNrvi&export=download
Resolving drive.google.com (drive.google.com)... 173.194.193.113, 173.194.193.102, 173.194.193.139, ...
Connecting to drive.google.com (drive.google.com)|173.194.193.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-70-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qj71p43ejqtfpev0mlio6d3vc7301ehu/1619348850000/14577798666193718332/*/1BgY3iHSZ0ImXc6atr61YQRcloxDwNrvi?e=download [following]
--2021-04-25 11:07:51--  https://doc-0s-70-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qj71p43ejqtfpev0mlio6d3vc7301ehu/1619348850000/14577798666193718332/*/1BgY3iHSZ0ImXc6atr61YQRcloxDwNrvi?e=download
Resolving doc-0s-70-docs.googleusercontent.com (doc-0s-70-docs.googleusercontent.com)... 172.217.219.132, 2607:f8b0:4001:c13::84
Connecting to doc-0s-70-docs.googleusercontent.co

In [121]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [140]:
def create_all_links(all_data):

  result = []
  cnt = len(all_data)

  for i in range(cnt):
    for j in range(i + 1, cnt):
      if (all_data[i] < all_data[j]):
        result.append( ((all_data[i], all_data[j]), 0))
      else:
        result.append( ((all_data[j], all_data[i]), 0))

  return result

def calc_cc(key, value):

  result = []
  cnt = len(value)

  for i in range(cnt):
    for j in range(i + 1, cnt):
      if (value[i] < value[j]):
        result.append( ((value[i], value[j]), key))
      else:
        result.append( ((value[j], value[i]), key))

  return result

def calc_car(key, value):

  result = []

  cnt = len(value)

  for i in range(cnt):
    for j in range(i + 1, cnt):
      for k in range(j +1, cnt):
        
        if (key < value[k]):
          first_data = (key, value[k])
        else:
          first_data = (value[k], key)

        if (value[i] < value[j]):
          second_data = (value[i], value[j])
        else:
          second_data = (value[j], value[i])
    
        result.append( (second_data , (first_data, 1)))

        if (key < value[j]):
          first_data = (key, value[j])
        else:
          first_data = (value[j], key)

        if (value[i] < value[k]):
          second_data = (value[i], value[k])
        else:
          second_data = (value[k], value[i])
    
        result.append( (second_data , (first_data, 1)))
        
        if (key < value[i]):
          first_data = (key, value[i])
        else:
          first_data = (value[i], key)

        if (value[k] < value[j]):
          second_data = (value[k], value[j])
        else:
          second_data = (value[j], value[k])
    
        result.append( (second_data , (first_data, 1)))
  return result

def Convert(lst):
  result = {}

  for item in lst:
    if (item[0] in result):
      result[item[0]] = result[item[0]] + 1
    else:
      result[item[0]] = 1

  return result

def remove_only_one_time_appers(dic):
  return sum(y / 2 for x,y in dic.items() if y> 1)

def car_grouped_mix(key, value):
  aggrigated = remove_only_one_time_appers(Convert(value))
  return (key, aggrigated)

In [ ]:
df = spark.sparkContext.textFile("paper_graph_adj_list.txt")
nodes = spark.sparkContext.textFile("paper_graph_node_list.txt")

temp_var = df.map(lambda k: k.split(" "))
temp_nodes = nodes.map(lambda k: k.split(" "))

all_links = temp_nodes.flatMap(lambda k : create_all_links(k))
data = temp_var.map(lambda k : (k[0], k[1::]))

cc_reduce = data.flatMap(lambda x: calc_cc(x[0], x[1]))
data_cn = cc_reduce.groupByKey().map(lambda x : (x[0], len(list(x[1]))))

car_reduce = data.flatMap(lambda x: calc_car(x[0], x[1]))
grouped = car_reduce.groupByKey().map(lambda x : (x[0], list(x[1])))

car_grouped_reduce = grouped.map(lambda x: (car_grouped_mix(x[0], x[1])))

joined_data = car_grouped_reduce.union(all_links)
join_by_all_links = joined_data.reduceByKey(lambda x, y: x + y)

car_for_all_links = join_by_all_links.union(data_cn).reduceByKey(lambda x, y: x * y)
car_for_all_links.collect()